In [3]:
import pandas as pd 
import numpy as np
import time
import requests

### Historical data from the odds api, but i have to pay. Worst case ill pay, if im not able to scrap from betting pros

In [ ]:
import requests
base_url = "https://api.the-odds-api.com/v4/sports/basketball_nba/events/"
API_KEY = '8aa8bb1fc5d56c98b45d6f3f58beab70'
Sport = 'basketball_nba'
Regions = 'us'
Market = 'h2h,odds'
Odds_format = 'american'
Date_format = 'iso' 
date = '2021-10-22T22:45:00Z'
id = '34e829d65b6a0cbb49573338a86772ec'

url = (
    f"https://api.the-odds-api.com/v4/historical/sports/{Sport}/events"
    f"?apiKey={API_KEY}"
    f"&date={date}"
    f"&regions={Regions}"
)

response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to get events: {response.status_code}, {response.text}")
else:
    events = response.json()
    for event in events:
        print(event['id'], event['commence_time'], event['home_team'], event['away_team'])

### Grabs players odds for the day(US all boookmakers, DFS is prizepicks and underdogs)

In [1]:
from NBAPropFinder.NBAPropFinder import NBAPropFinder

nba_props = NBAPropFinder()
prizePicks = nba_props.dataframe
prizePicks.to_csv('PROPS_DATA/Playoffs_DFS(6_12_25).csv')

Scraping Odds API...
Organizing Data...


### Single Bets from bookmakers that dont include prizePicks or UnderDogs

In [4]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
all_bookmakers = pd.read_csv('./PROPS_DATA/Playoffs_US(6_12_25).csv')
prop_type = all_bookmakers['CATEGORY'].unique()
print(prop_type)
all_bookmakers

['player_points' 'player_rebounds' 'player_assists' 'player_threes'
 'player_blocks' 'player_steals' 'player_rebounds_assists'
 'player_turnovers' 'player_blocks_steals']


,Unnamed: 0,BOOKMAKER,CATEGORY,NAME,OVER/UNDER,LINE,ODDS
0,0,FanDuel,player_points,Alex Caruso,Over,9.5,-112
1,1,FanDuel,player_points,Alex Caruso,Under,9.5,-118
2,2,FanDuel,player_points,Isaiah Hartenstein,Over,5.5,-132
3,3,FanDuel,player_points,Isaiah Hartenstein,Under,5.5,100
4,4,FanDuel,player_points,Obi Toppin,Over,9.5,-104
...,...,...,...,...,...,...,...
1481,1481,DraftKings,player_blocks_steals,Andrew Nembhard,Under,1.5,-135
1482,1482,DraftKings,player_blocks_steals,Pascal Siakam,Over,1.5,105
1483,1483,DraftKings,player_blocks_steals,Pascal Siakam,Under,1.5,-135
1484,1484,DraftKings,player_blocks_steals,Isaiah Hartenstein,Over,1.5,140


In [12]:
import scipy.stats as stats

def single_bet(data,bookmakers,category ='player_points', stat_line = 'PTS'):  
    Props = bookmakers[['NAME', 'BOOKMAKER', 'CATEGORY','LINE','OVER/UNDER', 'ODDS']].loc[bookmakers['CATEGORY'] == category]
    results = []

    for idx, row in Props.iterrows():
        name = row['NAME']
        bookmaker = row['BOOKMAKER']
        category = row['CATEGORY']
        line = row['LINE']
        over_under = row['OVER/UNDER']
        odds = row['ODDS']

        # Filter player data
        df = data.loc[data['PLAYER_NAME'] == name]
        if df.empty:
            continue

        # Mean and std dev of actual results
        mean = round(df[stat_line].mean(), 2)
        std = round(df[stat_line].std(), 2)
        if std == 0:
            continue

        # Z-score-based probability
        z_score = (line - mean) / std
        norm_prob = 1 - stats.norm.cdf(z_score)
        if over_under == "Under":
            norm_prob = 1 - norm_prob

        # Monte Carlo simulation
        sim_points = np.random.normal(mean, std, 10000)
        sim_prob = np.mean(sim_points > line)
        if over_under == "Under":
            sim_prob = 1 - sim_prob

        # Weighted average of sim and norm prob
        final_prob = 0.8 * sim_prob + 0.2 * norm_prob
        
        # EV calculation
        stake = 100
        profit = (odds / 100) * stake if odds > 0 else (100 / abs(odds)) * stake
        payout = stake + profit
        ev = (final_prob * profit) - ((1 - final_prob) * stake)

        # Kelly Criterion
        def kelly_criterion(prob, payout, stake):
            net_profit = payout - stake
            loss_prob = 1 - prob
            kelly = (net_profit * prob - loss_prob * stake) / net_profit
            return max(0, round(kelly, 4))

        kelly = kelly_criterion(final_prob, payout, stake)

        results.append({
            'NAME': name,
            'BOOKMAKER': bookmaker,
            'CATEGORY': category,
            'LINE': line,
            'OVER/UNDER': over_under,
            'ODDS': odds,
            'SIM PROB': round(final_prob, 3),
            'EV': round(ev, 2),
            'KELLY CRITERION': kelly,
        })

    return pd.DataFrame(results).sort_values(by='EV', ascending=False).reset_index(drop=True)


In [28]:
# Dictionary mapping prop categories to their stat columns
prop_mapping = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    'player_threes': 'FG3M',
    'player_blocks': 'BLK',
    'player_steals': 'STL',
    'player_turnovers': 'TOV',
}

all_results = []

for category, stat in prop_mapping.items():
    print(f"Processing {category}...")
    results = single_bet(data, all_bookmakers, category=category, stat_line=stat)
    all_results.append(results)

combined_results = pd.concat(all_results, ignore_index=True)

final_results = combined_results.sort_values(by='KELLY CRITERION', ascending=False).reset_index(drop=True)

print("\nTop 15 highest EV bets across all prop types:")
final_results.head(15)

# Save to CSV
# final_results.to_csv('PROPS_EV/SingleBets_AllProps(6_12_25).csv')

Processing player_points...
Processing player_rebounds...
Processing player_assists...
Processing player_threes...
Processing player_blocks...
Processing player_steals...
Processing player_turnovers...

Top 15 highest EV bets across all prop types:


,NAME,BOOKMAKER,CATEGORY,LINE,OVER/UNDER,ODDS,SIM PROB,EV,KELLY CRITERION
0,T.J. McConnell,Bovada,player_blocks,0.5,Under,-750,0.974,10.44,0.7831
1,Ben Sheppard,Bovada,player_blocks,0.5,Under,-900,0.966,7.37,0.6636
2,Pascal Siakam,Bovada,player_rebounds,8.5,Under,-215,0.886,29.74,0.6395
3,Pascal Siakam,Bovada,player_rebounds,8.5,Under,-215,0.882,29.24,0.6286
4,Isaiah Hartenstein,BetRivers,player_points,5.5,Over,-127,0.794,41.99,0.5333
5,Isaiah Hartenstein,FanDuel,player_points,5.5,Over,-132,0.785,37.98,0.5013
6,Shai Gilgeous-Alexander,Bovada,player_points,38.5,Under,-275,0.860,17.23,0.4737
7,Shai Gilgeous-Alexander,Bovada,player_points,37.5,Under,-230,0.836,19.89,0.4575
8,Pascal Siakam,DraftKings,player_turnovers,1.5,Under,135,0.687,61.44,0.4551
9,Pascal Siakam,BetOnline.ag,player_rebounds,7.5,Under,-130,0.761,34.62,0.4501


## PrizePicks & Underdog
## 2 leg EVs since we cant do single bets on those platforms

### Functions used to grab the ev and kelly

In [5]:
from scipy.stats import norm

def zscore_prob(mean, std, line, side="over"):
    z = (line - mean) / std
    prob = 1 - norm.cdf(z)
    return prob if side == "over" else 1 - prob

def kelly_criterion(probability, payout, stake, kelly_fraction=1.0):
    netProfit = payout - stake
    probabilityOfLoss = 1 - probability
    kelly = (netProfit * probability - probabilityOfLoss) / netProfit
    return max(0, round(kelly * kelly_fraction, 4))

def get_combined_stat_values(data, player_name, stat_line):
    """Helper function to get stat values for single or combined stats"""
    df = data.loc[data['PLAYER_NAME'] == player_name]
    
    if df.empty:
        return pd.Series()
    
    if '+' in stat_line:
        stat_parts = stat_line.split('+')
        return df[stat_parts].sum(axis=1)
    else:
        return df[stat_line]

def prizePicksPairsEV(data, prizePicks, propDict, simulations=10000, stake=100, payout=300, weight=0.8):
    def monte_carlo_sim(name1, name2, data, prizePicks1, prizePicks2, simulations, stake, payout, weight, stat_line1, stat_line2):
        df1 = data.loc[data['PLAYER_NAME'] == name1]
        df2 = data.loc[data['PLAYER_NAME'] == name2]

        if df1.empty or df2.empty:
            return None

        line1 = prizePicks1.loc[prizePicks1['NAME'] == name1, 'LINE'].values[0]
        line2 = prizePicks2.loc[prizePicks2['NAME'] == name2, 'LINE'].values[0]

        stat1_values = get_combined_stat_values(data, name1, stat_line1)
        stat2_values = get_combined_stat_values(data, name2, stat_line2)
        
        if len(stat1_values) == 0 or len(stat2_values) == 0:
            return None

        # Calculate mean and std 
        mean1, std1 = stat1_values.mean(), stat1_values.std()
        mean2, std2 = stat2_values.mean(), stat2_values.std()
        
        if std1 == 0 or std2 == 0:
            return None

        # Monte Carlo simulation using normal distribution
        sim1 = np.random.normal(mean1, std1, simulations)
        sim2 = np.random.normal(mean2, std2, simulations)

        over1, under1 = sim1 > line1, sim1 <= line1
        over2, under2 = sim2 > line2, sim2 <= line2

        mc_probs = {
            'OVER/OVER': np.mean(over1 & over2),
            'UNDER/UNDER': np.mean(under1 & under2),
            'OVER/UNDER': np.mean(over1 & under2),
            'UNDER/OVER': np.mean(under1 & over2),
        }

        z_over1 = zscore_prob(mean1, std1, line1, side='over')
        z_under1 = 1 - z_over1
        z_over2 = zscore_prob(mean2, std2, line2, side='over')
        z_under2 = 1 - z_over2

        z_probs = {
            'OVER/OVER': z_over1 * z_over2,
            'UNDER/UNDER': z_under1 * z_under2,
            'OVER/UNDER': z_over1 * z_under2,
            'UNDER/OVER': z_under1 * z_over2,
        }

        final_probs = {
            k: weight * mc_probs[k] + (1 - weight) * z_probs[k]
            for k in mc_probs
        }

        evs = {k: round((final_probs[k] * payout) - stake, 2) for k in final_probs}

        return {
            'OVER/OVER': {'prob': round(final_probs['OVER/OVER'], 4), 'ev': evs['OVER/OVER']},
            'UNDER/UNDER': {'prob': round(final_probs['UNDER/UNDER'], 4), 'ev': evs['UNDER/UNDER']},
            'OVER/UNDER': {'prob': round(final_probs['OVER/UNDER'], 4), 'ev': evs['OVER/UNDER']},
            'UNDER/OVER': {'prob': round(final_probs['UNDER/OVER'], 4), 'ev': evs['UNDER/OVER']}
        }

    all_pairs = []
    seen = set()

    # Iterate through all categories for player 1
    for cat1 in propDict:
        print(f"Processing {cat1}...")
        prizePicks1 = prizePicks[prizePicks['CATEGORY'] == cat1]
        stat_line1 = propDict[cat1]
        
        # Compare against all categories (including same category)
        for cat2 in propDict:
            print(f"Processing {cat2}...")
            prizePicks2 = prizePicks[prizePicks['CATEGORY'] == cat2]
            stat_line2 = propDict[cat2]
            
            for name1 in prizePicks1['NAME'].unique():
                for name2 in prizePicks2['NAME'].unique():
                    if name1 == name2:
                        continue
                        
                    # Creates unique keys
                    combo_key = tuple(sorted([
                        (name1, cat1),
                        (name2, cat2)
                    ]))
                    
                    if combo_key in seen:
                        continue
                    seen.add(combo_key)

                    sim_result = monte_carlo_sim(
                        name1, name2, data, 
                        prizePicks1, prizePicks2,
                        simulations, stake, payout, weight,
                        stat_line1, stat_line2
                    )
                    
                    if sim_result is None:
                        continue

                    evs = {
                        'OVER/OVER': sim_result['OVER/OVER']['ev'],
                        'UNDER/UNDER': sim_result['UNDER/UNDER']['ev'],
                        'OVER/UNDER': sim_result['OVER/UNDER']['ev'],
                        'UNDER/OVER': sim_result['UNDER/OVER']['ev']
                    }
                    
                    probs = {
                        'OVER/OVER': sim_result['OVER/OVER']['prob'],
                        'UNDER/UNDER': sim_result['UNDER/UNDER']['prob'],
                        'OVER/UNDER': sim_result['OVER/UNDER']['prob'],
                        'UNDER/OVER': sim_result['UNDER/OVER']['prob']
                    }

                    best_type = max(evs, key=evs.get)
                    best_ev = evs[best_type]
                    best_prob = probs[best_type]

                    all_pairs.append({
                        'PLAYER 1': name1,
                        'CATEGORY 1': stat_line1,
                        'PLAYER 1 LINE': prizePicks1.loc[prizePicks1['NAME'] == name1, 'LINE'].values[0],
                        'PLAYER 2': name2,
                        'CATEGORY 2': stat_line2,
                        'PLAYER 2 LINE': prizePicks2.loc[prizePicks2['NAME'] == name2, 'LINE'].values[0],
                        'TYPE': best_type,
                        'EV': round(best_ev, 2),
                        'PROBABILITY': round(best_prob, 4),
                        'KELLY CRITERION': kelly_criterion(best_prob, payout, stake)
                    })

    return pd.DataFrame(all_pairs)

In [ ]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
pp_data = pd.read_csv('PROPS_DATA/Playoffs_DFS(6_12_25).csv')
prizePicks = pp_data[['NAME', 'BOOKMAKER', 'CATEGORY','LINE','OVER/UNDER', 'ODDS']].loc[pp_data['BOOKMAKER'] == 'PrizePicks']
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    'player_threes': 'FG3M',
    'player_blocks': 'BLK',
    'player_steals': 'STL',
    'player_field_goals': 'FGM',
    'player_frees_made': 'FTM',
    'player_frees_attempts': 'FTA',
    'player_points_rebounds_assists': 'PTS+REB+AST',
    'player_points_rebounds': 'PTS+REB',
    'player_points_assists': 'PTS+AST',
    'player_rebounds_assists': 'REB+AST',
    'player_turnovers': 'TOV',
}

# Run the analysis
results = prizePicksPairsEV(data, pp_data, propDict)

# Show top 10 highest EV combinations
results.sort_values(by='KELLY CRITERION', ascending=False).head(10)

Processing player_points...
Processing player_points...
Processing player_rebounds...
Processing player_assists...
Processing player_threes...
Processing player_blocks...
Processing player_steals...
Processing player_field_goals...
Processing player_frees_made...
Processing player_frees_attempts...
Processing player_points_rebounds_assists...
Processing player_points_rebounds...
Processing player_points_assists...
Processing player_rebounds_assists...
Processing player_turnovers...
Processing player_rebounds...
Processing player_points...
Processing player_rebounds...
Processing player_assists...
Processing player_threes...
Processing player_blocks...
Processing player_steals...
Processing player_field_goals...
Processing player_frees_made...
Processing player_frees_attempts...
Processing player_points_rebounds_assists...
Processing player_points_rebounds...
Processing player_points_assists...
Processing player_rebounds_assists...
Processing player_turnovers...
Processing player_assist

,PLAYER 1,CATEGORY 1,PLAYER 1 LINE,PLAYER 2,CATEGORY 2,PLAYER 2 LINE,TYPE,EV,PROBABILITY,KELLY CRITERION
2965,Pascal Siakam,REB,7.5,Isaiah Hartenstein,FGM,2.5,UNDER/OVER,75.22,0.5841,0.5820
359,Isaiah Hartenstein,PTS,6.0,Pascal Siakam,REB,7.5,OVER/UNDER,69.60,0.5653,0.5631
875,Isaiah Joe,PTS,2.5,Isaiah Hartenstein,FGM,2.5,OVER/OVER,68.65,0.5622,0.5600
3578,Pascal Siakam,REB,7.5,Isaiah Hartenstein,PTS+REB,13.0,UNDER/OVER,66.21,0.5540,0.5518
164,Isaiah Joe,PTS,2.5,Pascal Siakam,REB,7.5,OVER/UNDER,63.87,0.5462,0.5439
3772,Pascal Siakam,REB,7.5,Isaiah Hartenstein,PTS+AST,8.0,UNDER/OVER,62.67,0.5422,0.5399
6922,Shai Gilgeous-Alexander,FGM,12.0,Isaiah Hartenstein,FGM,2.5,UNDER/OVER,62.50,0.5417,0.5394
15,Isaiah Joe,PTS,2.5,Isaiah Hartenstein,PTS,6.0,OVER/OVER,62.06,0.5402,0.5379
2964,Pascal Siakam,REB,7.5,Shai Gilgeous-Alexander,FGM,12.0,UNDER/UNDER,61.21,0.5374,0.5351
7603,Isaiah Hartenstein,FGM,2.5,Alex Caruso,REB+AST,6.5,OVER/UNDER,60.71,0.5357,0.5334


In [14]:
results.to_csv('PROPS_EV/PrizePicksPairs(6_12_25).csv')

## 3 leg EVs for prizepicks and underdogs

In [18]:
def prizePicksTriosEV(data, prizePicks, propDict, simulations=10000, stake=100, payout=600, weight=0.8):
    def monte_carlo_sim_trio(name1, name2, name3, data, prizePicks1, prizePicks2, prizePicks3, simulations, stake, payout, weight, stat_line1, stat_line2, stat_line3):
        df1 = data.loc[data['PLAYER_NAME'] == name1]
        df2 = data.loc[data['PLAYER_NAME'] == name2]
        df3 = data.loc[data['PLAYER_NAME'] == name3]

        line1 = prizePicks1.loc[prizePicks1['NAME'] == name1, 'LINE'].values[0]
        line2 = prizePicks2.loc[prizePicks2['NAME'] == name2, 'LINE'].values[0]
        line3 = prizePicks3.loc[prizePicks3['NAME'] == name3, 'LINE'].values[0]

        stat1_values = get_combined_stat_values(data, name1, stat_line1)
        stat2_values = get_combined_stat_values(data, name2, stat_line2)
        stat3_values = get_combined_stat_values(data, name3, stat_line3)
        
        mean1, std1 = stat1_values.mean(), stat1_values.std()
        mean2, std2 = stat2_values.mean(), stat2_values.std()
        mean3, std3 = stat3_values.mean(), stat3_values.std()

        if std1 == 0 or std2 == 0 or std3 == 0:
            return None

        sim1 = np.random.normal(mean1, std1, simulations)
        sim2 = np.random.normal(mean2, std2, simulations)
        sim3 = np.random.normal(mean3, std3, simulations)

        over1, under1 = sim1 > line1, sim1 <= line1
        over2, under2 = sim2 > line2, sim2 <= line2
        over3, under3 = sim3 > line3, sim3 <= line3

        mc_probs = {
            'OVER/OVER/OVER': np.mean(over1 & over2 & over3),
            'OVER/OVER/UNDER': np.mean(over1 & over2 & under3),
            'OVER/UNDER/OVER': np.mean(over1 & under2 & over3),
            'OVER/UNDER/UNDER': np.mean(over1 & under2 & under3),
            'UNDER/OVER/OVER': np.mean(under1 & over2 & over3),
            'UNDER/OVER/UNDER': np.mean(under1 & over2 & under3),
            'UNDER/UNDER/OVER': np.mean(under1 & under2 & over3),
            'UNDER/UNDER/UNDER': np.mean(under1 & under2 & under3),
        }

        z_over1 = zscore_prob(mean1, std1, line1, side='over')
        z_under1 = 1 - z_over1
        z_over2 = zscore_prob(mean2, std2, line2, side='over')
        z_under2 = 1 - z_over2
        z_over3 = zscore_prob(mean3, std3, line3, side='over')
        z_under3 = 1 - z_over3

        z_probs = {
            'OVER/OVER/OVER': z_over1 * z_over2 * z_over3,
            'OVER/OVER/UNDER': z_over1 * z_over2 * z_under3,
            'OVER/UNDER/OVER': z_over1 * z_under2 * z_over3,
            'OVER/UNDER/UNDER': z_over1 * z_under2 * z_under3,
            'UNDER/OVER/OVER': z_under1 * z_over2 * z_over3,
            'UNDER/OVER/UNDER': z_under1 * z_over2 * z_under3,
            'UNDER/UNDER/OVER': z_under1 * z_under2 * z_over3,
            'UNDER/UNDER/UNDER': z_under1 * z_under2 * z_under3,
        }

        probs = {
            k: weight * mc_probs[k] + (1 - weight) * z_probs[k]
            for k in mc_probs
        }

        evs = {k: round((probs[k] * payout) - stake, 2) for k in probs}

        return {
            combo: {'prob': round(probs[combo], 4), 'ev': evs[combo]}
            for combo in probs
        }

    all_trios = []
    seen = set()

    # Iterate through all categories for player 1
    for cat1 in propDict:
        print(f"Processing {cat1}...")
        prizePicks1 = prizePicks[prizePicks['CATEGORY'] == cat1]
        stat_line1 = propDict[cat1]
        
        # Compare against all categories for player 2
        for cat2 in propDict:
            print(f"Processing {cat2}...")
            prizePicks2 = prizePicks[prizePicks['CATEGORY'] == cat2]
            stat_line2 = propDict[cat2]
            
            # Compare against all categories for player 3
            for cat3 in propDict:
                print(f"Processing {cat3}...")
                prizePicks3 = prizePicks[prizePicks['CATEGORY'] == cat3]
                stat_line3 = propDict[cat3]
                
                for name1 in prizePicks1['NAME'].unique():
                    for name2 in prizePicks2['NAME'].unique():
                        for name3 in prizePicks3['NAME'].unique():
                            # Skip if any players are the same
                            if name1 == name2 or name1 == name3 or name2 == name3:
                                continue
                                
                            # Create unique key for this combination
                            combo_key = tuple(sorted([
                                (name1, cat1),
                                (name2, cat2),
                                (name3, cat3)
                            ]))
                            
                            if combo_key in seen:
                                continue
                            seen.add(combo_key)

                            sim_result = monte_carlo_sim_trio(
                                name1, name2, name3, data, 
                                prizePicks1, prizePicks2, prizePicks3,
                                simulations, stake, payout, weight,
                                stat_line1, stat_line2, stat_line3
                            )
                            
                            if sim_result is None:
                                continue

                            evs = {combo: sim_result[combo]['ev'] for combo in sim_result}
                            probs = {combo: sim_result[combo]['prob'] for combo in sim_result}

                            best_type = max(evs, key=evs.get)
                            best_ev = evs[best_type]
                            best_prob = probs[best_type]

                            all_trios.append({
                                'PLAYER 1': name1,
                                'CATEGORY 1': stat_line1,
                                'PLAYER 1 LINE': prizePicks1.loc[prizePicks1['NAME'] == name1, 'LINE'].values[0],
                                'PLAYER 2': name2,
                                'CATEGORY 2': stat_line2,
                                'PLAYER 2 LINE': prizePicks2.loc[prizePicks2['NAME'] == name2, 'LINE'].values[0],
                                'PLAYER 3': name3,
                                'CATEGORY 3': stat_line3,
                                'PLAYER 3 LINE': prizePicks3.loc[prizePicks3['NAME'] == name3, 'LINE'].values[0],
                                'TYPE': best_type,
                                'EV': round(best_ev, 2),
                                'PROBABILITY': round(best_prob, 4),
                                'KELLY CRITERION': kelly_criterion(best_prob, payout, stake)
                            })

    return pd.DataFrame(all_trios)

In [19]:
# Example usage
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    'player_threes': 'FG3M',
    'player_blocks': 'BLK',
    'player_steals': 'STL',
    'player_field_goals': 'FGM',
    'player_frees_made': 'FTM',
    'player_frees_attempts': 'FTA',
    'player_points_rebounds_assists': 'PTS+REB+AST',
    'player_points_rebounds': 'PTS+REB',
    'player_points_assists': 'PTS+AST',
    'player_rebounds_assists': 'REB+AST',
    'player_turnovers': 'TOV',
}
# Run the 3-pick analysis (note the higher payout of 600 for 3-picks vs 300 for 2-picks)
trio_results = prizePicksTriosEV(data, prizePicks, propDict, payout=600)
trio_results.sort_values('KELLY CRITERION', ascending=False).reset_index(drop=True)
trio_results
# Save results
# trio_results.to_csv('PROPS_EV/PrizePicksTrios(6_12_25).csv')

Processing player_points...
Processing player_points...
Processing player_points...
Processing player_rebounds...
Processing player_assists...
Processing player_threes...
Processing player_blocks...
Processing player_steals...
Processing player_field_goals...
Processing player_frees_made...
Processing player_frees_attempts...
Processing player_points_rebounds_assists...
Processing player_points_rebounds...
Processing player_points_assists...
Processing player_rebounds_assists...
Processing player_turnovers...
Processing player_rebounds...
Processing player_points...
Processing player_rebounds...
Processing player_assists...
Processing player_threes...
Processing player_blocks...
Processing player_steals...
Processing player_field_goals...
Processing player_frees_made...
Processing player_frees_attempts...
Processing player_points_rebounds_assists...
Processing player_points_rebounds...
Processing player_points_assists...
Processing player_rebounds_assists...
Processing player_turnovers

,PLAYER 1,CATEGORY 1,PLAYER 1 LINE,PLAYER 2,CATEGORY 2,PLAYER 2 LINE,PLAYER 3,CATEGORY 3,PLAYER 3 LINE,TYPE,EV,PROBABILITY,KELLY CRITERION
0,Isaiah Joe,PTS,2.5,Myles Turner,PTS,13.5,T.J. McConnell,PTS,7.5,OVER/OVER/OVER,55.73,0.2596,0.2581
1,Isaiah Joe,PTS,2.5,Myles Turner,PTS,13.5,Luguentz Dort,PTS,8.5,OVER/OVER/UNDER,38.05,0.2301,0.2286
2,Isaiah Joe,PTS,2.5,Myles Turner,PTS,13.5,Aaron Wiggins,PTS,5.5,OVER/OVER/OVER,46.79,0.2447,0.2432
3,Isaiah Joe,PTS,2.5,Myles Turner,PTS,13.5,Jalen Williams,PTS,21.5,OVER/OVER/UNDER,52.38,0.2540,0.2525
4,Isaiah Joe,PTS,2.5,Myles Turner,PTS,13.5,Andrew Nembhard,PTS,10.5,OVER/OVER/OVER,69.82,0.2830,0.2816
...,...,...,...,...,...,...,...,...,...,...,...,...,...
300899,Tyrese Haliburton,REB+AST,14.0,Alex Caruso,REB+AST,6.5,T.J. McConnell,TOV,1.5,OVER/UNDER/OVER,33.48,0.2225,0.2209
300900,Tyrese Haliburton,REB+AST,14.0,Isaiah Hartenstein,REB+AST,8.5,T.J. McConnell,TOV,1.5,OVER/OVER/OVER,22.72,0.2045,0.2029
300901,Shai Gilgeous-Alexander,REB+AST,12.5,Alex Caruso,REB+AST,6.5,T.J. McConnell,TOV,1.5,UNDER/UNDER/OVER,14.29,0.1905,0.1889
300902,Shai Gilgeous-Alexander,REB+AST,12.5,Isaiah Hartenstein,REB+AST,8.5,T.J. McConnell,TOV,1.5,UNDER/OVER/OVER,3.45,0.1724,0.1707


⚙️ Hybrid Approach Recommended
For NBA prop betting, a hybrid workflow often works best:
- Train an ML model to predict the expected prop value using features like usage rate, matchup context, rest, etc.
- Simulate multiple game scenarios via Monte Carlo, using your ML model’s predictive distribution (e.g., add residual noise or use a Bayesian framework).
- Bootstrap on your historical residuals to calibrate confidence intervals around those predictions.

This combines:
ML’s ability to capture complex relationships,
Monte Carlo’s capacity for modeling variability and outputting a full prob. distribution,
Bootstrap’s non-parametric estimate of uncertainty—all necessary to assess value and risk correctly 

🗓️ Practical Takeaway
Need a full probability distribution for value + risk? MC sim driven by your ML model is essential.


📌 Final Recommendation
Use Machine Learning to predict the mean prop, then Monte Carlo to simulate possible outcomes (using those ML predictions with added variance), and Bootstrap to quantify your confidence in those outcomes. This hybrid setup offers the most accurate, actionable probabilities for NBA prop betting.
